In [ ]:
!pip install -U flwr["simulation"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 KB 548.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.1/128.1 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.0/202.0 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 KB 4.6 MB/s eta 0:00:00
  Created wheel for gpustat: filename=gpustat-1.0.0b1-py3-none-any.whl size=15979 sha256=0118860ed10409b8ab7d983c07f0fd0c661e02ef629d3cd4baaa53b9fe38e814
  Stored in directory: /root/.cache/pip/wheels/1a/16/e2/3e2437fba4c4b6a97a97bd96fce5d14e66cff5c4966fb1cc8c
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=

In [ ]:
import os
import math

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import flwr as fl
import tensorflow as tf

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val

    def get_parameters(self):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=1, verbose=2)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=2)
        return loss, len(self.x_val), {"accuracy": acc}

In [ ]:
import pandas as pd
df=pd.read_csv('../input/emotion-final-2/Emotion_final csv.csv')

In [ ]:
df.head()

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
category = pd.get_dummies(df.Emotion)
df_new = pd.concat([df, category], axis=1)
df_new = df_new.drop(columns='Emotion')
df_new

,Text,anger,fear,happy,love,sadness,surprise
0,i didnt feel humiliated,0,0,0,0,1,0
1,i can go from feeling so hopeless to so damned...,0,0,0,0,1,0
2,im grabbing a minute to post i feel greedy wrong,1,0,0,0,0,0
3,i am ever feeling nostalgic about the fireplac...,0,0,0,1,0,0
4,i am feeling grouchy,1,0,0,0,0,0
...,...,...,...,...,...,...,...
21454,Melissa stared at her friend in dism,0,1,0,0,0,0
21455,Successive state elections have seen the gover...,0,1,0,0,0,0
21456,Vincent was irritated but not dismay,0,1,0,0,0,0
21457,Kendall-Hume turned back to face the dismayed ...,0,1,0,0,0,0


In [ ]:
text = df_new['Text'].values
label = df_new[['anger', 'fear', 'happy', 'love', 'sadness', 'surprise']].values

In [ ]:
from sklearn.model_selection import train_test_split
text_train, text_test, label_train, label_test = train_test_split(text, label, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=2000, oov_token='x')
tokenizer.fit_on_texts(text_train) 
tokenizer.fit_on_texts(text_test)
 
sekuens_train = tokenizer.texts_to_sequences(text_train)
sekuens_test = tokenizer.texts_to_sequences(text_test)
 
padded_train = pad_sequences(sekuens_train) 
padded_test = pad_sequences(sekuens_test)

In [ ]:
def client_fn(cid: str) -> fl.client.Client:
    # Create model
    model = tf.keras.models.Sequential(
        [
    tf.keras.layers.Embedding(input_dim=2000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
        ]
    )
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

    # Load data partition (divide MNIST into NUM_CLIENTS distinct partitions)
    #(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
    partition_size = math.floor(len(padded_train) / NUM_CLIENTS)
    idx_from, idx_to = int(cid) * partition_size, (int(cid) + 1) * partition_size
    x_train_cid = padded_train[idx_from:idx_to] / 255.0
    y_train_cid = label_train[idx_from:idx_to]

    # Use 10% of the client's training data for validation
    split_idx = math.floor(len(padded_train) * 0.9)
    x_train_cid, y_train_cid = x_train_cid[:split_idx], y_train_cid[:split_idx]
    x_val_cid, y_val_cid = x_train_cid[split_idx:], y_train_cid[split_idx:]

    # Create and return client
    return FlowerClient(model, x_train_cid, y_train_cid, x_val_cid, y_val_cid)

In [ ]:
pip install -U flwr["simulation"]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
NUM_CLIENTS = 100

# Create FedAvg strategy
strategy=fl.server.strategy.FedAvg(
        fraction_fit=0.5,  #.1 Sample 50% of available clients for training
        fraction_eval=0.25,  #.05 Sample 50% of available clients for evaluation
        min_fit_clients=1,  # Never sample less than 10 clients for training
        min_eval_clients=1,  # Never sample less than 5 clients for evaluation
        min_available_clients=int(NUM_CLIENTS * 0.50),  # Wait until at least 5 clients are available
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=5,
    strategy=strategy,
)

INFO flower 2022-04-30 18:04:58,834 | app.py:147 | Ray initialized with resources: {'node:172.19.2.2': 1.0, 'object_store_memory': 4913953996.0, 'CPU': 4.0, 'memory': 9827907995.0}
INFO flower 2022-04-30 18:04:58,837 | app.py:156 | Starting Flower simulation running: {'num_rounds': 5}
INFO flower 2022-04-30 18:04:58,838 | server.py:128 | Initializing global parameters
INFO flower 2022-04-30 18:04:58,839 | server.py:327 | Requesting initial parameters from one random client
INFO flower 2022-04-30 18:05:01,819 | server.py:330 | Received initial parameters from one random client
INFO flower 2022-04-30 18:05:01,820 | server.py:130 | Evaluating initial parameters
INFO flower 2022-04-30 18:05:01,822 | server.py:143 | FL starting
DEBUG flower 2022-04-30 18:05:01,823 | server.py:269 | fit_round: strategy sampled 50 clients (out of 100)


(launch_and_fit pid=190) 6/6 - 4s - loss: 1.7795 - accuracy: 0.3099
(launch_and_fit pid=191) 6/6 - 5s - loss: 1.7806 - accuracy: 0.2982
(launch_and_fit pid=189) 6/6 - 5s - loss: 1.7790 - accuracy: 0.3333
(launch_and_fit pid=192) 6/6 - 6s - loss: 1.7781 - accuracy: 0.3450
(launch_and_fit pid=190) 6/6 - 5s - loss: 1.7784 - accuracy: 0.3216
(launch_and_fit pid=191) 6/6 - 5s - loss: 1.7796 - accuracy: 0.3099
(launch_and_fit pid=192) 6/6 - 5s - loss: 1.7823 - accuracy: 0.2749
(launch_and_fit pid=189) 6/6 - 6s - loss: 1.7784 - accuracy: 0.3158
(launch_and_fit pid=190) 6/6 - 5s - loss: 1.7808 - accuracy: 0.3216
(launch_and_fit pid=191) 6/6 - 4s - loss: 1.7795 - accuracy: 0.2924
(launch_and_fit pid=192) 6/6 - 5s - loss: 1.7821 - accuracy: 0.3099
(launch_and_fit pid=189) 6/6 - 5s - loss: 1.7794 - accuracy: 0.2807
(launch_and_fit pid=190) 6/6 - 6s - loss: 1.7803 - accuracy: 0.3275
(launch_and_fit pid=191) 6/6 - 4s - loss: 1.7789 - accuracy: 0.2982
(launch_and_fit pid=192) 6/6 - 4s - loss: 1.7815

DEBUG flower 2022-04-30 18:06:13,941 | server.py:281 | fit_round received 50 results and 0 failures


(launch_and_fit pid=190) 6/6 - 3s - loss: 1.7837 - accuracy: 0.2865


DEBUG flower 2022-04-30 18:06:14,401 | server.py:215 | evaluate_round: strategy sampled 25 clients (out of 100)
(launch_and_evaluate pid=191) /opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:977: RuntimeWarning: divide by zero encountered in log10
(launch_and_evaluate pid=191)   numdigits = int(np.log10(self.target)) + 1
(launch_and_evaluate pid=189) /opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:977: RuntimeWarning: divide by zero encountered in log10
(launch_and_evaluate pid=189)   numdigits = int(np.log10(self.target)) + 1
(launch_and_evaluate pid=190) /opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:977: RuntimeWarning: divide by zero encountered in log10
(launch_and_evaluate pid=190)   numdigits = int(np.log10(self.target)) + 1
(launch_and_evaluate pid=192) /opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:977: RuntimeWarning: divide by zero encountered in log10
(launch_and_evaluate pid=192)   numdigits =

...

DEBUG flower 2022-04-30 18:09:52,471 | server.py:227 | evaluate_round received 0 results and 25 failures
DEBUG flower 2022-04-30 18:09:52,473 | server.py:269 | fit_round: strategy sampled 50 clients (out of 100)


(launch_and_fit pid=191) 6/6 - 4s - loss: 1.6920 - accuracy: 0.3041
(launch_and_fit pid=189) 6/6 - 4s - loss: 1.6082 - accuracy: 0.3450
(launch_and_fit pid=190) 6/6 - 5s - loss: 1.5902 - accuracy: 0.3392
(launch_and_fit pid=192) 6/6 - 5s - loss: 1.5908 - accuracy: 0.2865
(launch_and_fit pid=191) 6/6 - 4s - loss: 1.6292 - accuracy: 0.3041
(launch_and_fit pid=189) 6/6 - 5s - loss: 1.6657 - accuracy: 0.3450
(launch_and_fit pid=190) 6/6 - 5s - loss: 1.6138 - accuracy: 0.3041
(launch_and_fit pid=192) 6/6 - 5s - loss: 1.6693 - accuracy: 0.2924
(launch_and_fit pid=191) 6/6 - 5s - loss: 1.6329 - accuracy: 0.3567
(launch_and_fit pid=190) 6/6 - 4s - loss: 1.6205 - accuracy: 0.2865
(launch_and_fit pid=192) 6/6 - 4s - loss: 1.5859 - accuracy: 0.2573
(launch_and_fit pid=189) 6/6 - 5s - loss: 1.5646 - accuracy: 0.3275
(launch_and_fit pid=191) 6/6 - 4s - loss: 1.6158 - accuracy: 0.3041
(launch_and_fit pid=192) 6/6 - 5s - loss: 1.6152 - accuracy: 0.3158
(launch_and_fit pid=189) 6/6 - 4s - loss: 1.5982

DEBUG flower 2022-04-30 18:10:59,061 | server.py:281 | fit_round received 50 results and 0 failures
DEBUG flower 2022-04-30 18:10:59,237 | server.py:215 | evaluate_round: strategy sampled 25 clients (out of 100)


(launch_and_fit pid=190) 6/6 - 3s - loss: 1.6143 - accuracy: 0.3099


DEBUG flower 2022-04-30 18:11:02,696 | server.py:227 | evaluate_round received 0 results and 25 failures
INFO flower 2022-04-30 18:11:02,697 | server.py:182 | FL finished in 360.873941467
INFO flower 2022-04-30 18:11:02,699 | app.py:149 | app_fit: losses_distributed []
INFO flower 2022-04-30 18:11:02,700 | app.py:150 | app_fit: metrics_distributed {}
INFO flower 2022-04-30 18:11:02,700 | app.py:151 | app_fit: losses_centralized []
INFO flower 2022-04-30 18:11:02,702 | app.py:152 | app_fit: metrics_centralized {}
